We perform extractive text summarization using the TextRank algorithm which uses a graph-based ranking model which requires no training, using GloVe v1.2 pre-trained word vectors, and performing various natural language preprocessing using NLTK library.

Graph based ranking algorithms allow knowledge about the text as a whole and the relationship between different parts of a text to be used in making specific local ranking decisions.  It does so by taking into account global information recursively computed from the entire graph in order to evaluate the importance of a vertex within a graph, rather than relying only on local information.

Traditional word vector techniques depend on the distance or angle between pairs of word vectors to determine strength of a set  word representations.  Glove attempts to uncover more of the language structure by examining not only the scalar difference but various dimensions of difference.  It does this by examining the ratio of the co-occurance probability between pairings rather the just the probabilties themselves.  A weighted least squares regression is then applied to remove the noise.  Dimesionality reduction is applied to the co-occurance matrix to yield a lower dimensional matrix such that each vector represents a word.

In [1]:
import numpy as np
import pandas as pd
import nltk
import re

In [30]:
article = open("../Text-Summarizer/articles/nytimes article.txt", "r")

sentences = []
for line in article:
    sentences.append(line.rstrip())
sentences[:5]

['WASHINGTON — Defense Secretary Jim Mattis, whose experience and stability were widely seen as a balance to an unpredictable president, resigned on Thursday in protest of President Trump’s decision to withdraw 2,000 American troops from Syria.',
 '',
 'Mr. Trump announced Mr. Mattis’s resignation in two tweets Thursday evening, and said the retired four-star Marine general turned Pentagon chief will leave at the end of February.',
 '',
 'Officials said Mr. Mattis went to the White House on Thursday afternoon in a last attempt to convince Mr. Trump to keep American troops in Syria, where they have been fighting the Islamic State. He was rebuffed, and told the president that he was resigning as a result.']

In [3]:
print(len(sentences))

31


In [4]:
sentences = [s for s in sentences if len(s) != 0]

In [5]:
print(len(sentences))

18


In [6]:
from nltk.tokenize import sent_tokenize
new_sentences = []
for s in sentences:
  new_sentences.append(sent_tokenize(s))

new_sentences = [y for x in new_sentences for y in x] # flatten list

In [7]:
new_sentences[:5]

['WASHINGTON — Defense Secretary Jim Mattis, whose experience and stability were widely seen as a balance to an unpredictable president, resigned on Thursday in protest of President Trump’s decision to withdraw 2,000 American troops from Syria.',
 'Mr. Trump announced Mr. Mattis’s resignation in two tweets Thursday evening, and said the retired four-star Marine general turned Pentagon chief will leave at the end of February.',
 'Officials said Mr. Mattis went to the White House on Thursday afternoon in a last attempt to convince Mr. Trump to keep American troops in Syria, where they have been fighting the Islamic State.',
 'He was rebuffed, and told the president that he was resigning as a result.',
 'Hours later, the Pentagon released Mr. Mattis’s resignation letter, in which he implicitly criticized his commander in chief.']

In [8]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [9]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(new_sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [10]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jhsoo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [12]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [13]:
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [14]:
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [15]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [16]:
for i in range(len(sentence_vectors)):
    sentence_vectors[i] = sentence_vectors[i].reshape(1, 100)

In [17]:
print(len(sentence_vectors))

26


In [18]:
sim_mat = np.zeros([len(sentences), len(sentences)])

In [19]:
print(sim_mat.shape)

(18, 18)


In [20]:
from sklearn.metrics.pairwise import cosine_similarity


Cosine similarity computes the similarity between vectors based on the degree of orthogonality between vectors where a cosine of 1 is identical and a cosine of 0 is orthogonality.

In [21]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i], sentence_vectors[j])[0,0]

In [22]:
import networkx as nx

nx_graph = nx.from_numpy_matrix(sim_mat)
scores = nx.pagerank(nx_graph)

In [23]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [24]:
#takes 33% of the most relevant content from the original article
for i in range(int(len(sentence_vectors)*(1/3))):
    print(ranked_sentences[i][1], end="\n\n")

WASHINGTON — Defense Secretary Jim Mattis, whose experience and stability were widely seen as a balance to an unpredictable president, resigned on Thursday in protest of President Trump’s decision to withdraw 2,000 American troops from Syria.

Officials said Mr. Mattis went to the White House on Thursday afternoon in a last attempt to convince Mr. Trump to keep American troops in Syria, where they have been fighting the Islamic State. He was rebuffed, and told the president that he was resigning as a result.

“This is scary,” Senator Mark Warner of Virginia, the top Democrat on the Senate Intelligence Committee, said in a tweet. He called Mr. Mattis “an island of stability amidst the chaos of the Trump administration.”

Over the past six months, the president and the defense chief have also found themselves at odds over NATO policy, whether to resume large-scale military exercises with South Korea and, privately, whether Mr. Trump’s decision to withdraw the United States from the Iran 